<a href="https://colab.research.google.com/github/sril1485/CE888/blob/master/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
data = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/user_ratings.csv")

In [0]:
n_latent_factors = 2

user_ratings = data.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [4]:
latent_item_features


array([[0.741, 0.084],
       [0.707, 0.626],
       [0.126, 0.645],
       [0.298, 0.452],
       [0.913, 0.549]])

In [5]:
latent_user_preferences


array([[0.945, 0.325],
       [0.307, 0.251],
       [0.259, 0.49 ],
       [0.053, 0.903],
       [0.454, 0.793],
       [0.44 , 0.17 ],
       [0.593, 0.694],
       [0.812, 0.015],
       [0.387, 0.448],
       [0.841, 0.621]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=1000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)

In [12]:
sgd()

1.4367202141111393


In [13]:

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 7.798,  1.406, -0.497,  5.346,  4.079],
       [ 3.514,  2.531,  9.778,  5.879,  7.395],
       [ 7.873,  2.6  ,  5.721,  7.557,  7.575],
       [ 8.193,  2.937,  7.173,  8.287,  8.561],
       [ 1.478,  1.872,  8.369,  3.949,  5.475],
       [ 2.283,  1.449,  5.321,  3.462,  4.231],
       [ 3.316,  1.811,  6.181,  4.491,  5.286],
       [ 8.275,  2.2  ,  3.205,  6.968,  6.402],
       [ 3.717,  1.759,  5.504,  4.54 ,  5.134],
       [ 3.003,  1.38 ,  4.228,  3.592,  4.026]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns

In [15]:

comparison_data

,The Call of Cthulhu,Frankenstein,Dracula,Neuromancer,Space Odyssey
0,"(8.0, 7.798100588912573)","(2.0, 1.4060796490147416)","(nan, -0.49663903581604735)","(5.0, 5.346391210821107)","(4.0, 4.07903428107082)"
1,"(3.0, 3.513611084661923)","(2.0, 2.5309264604716892)","(nan, 9.777886726153431)","(7.0, 5.878802971151284)","(7.0, 7.394626443537732)"
2,"(9.0, 7.873225876245922)","(nan, 2.600054015272773)","(7.0, 5.720951164708522)","(8.0, 7.556619288602486)","(5.0, 7.575359969844888)"
3,"(nan, 8.19276267527872)","(nan, 2.937023274325186)","(7.0, 7.173152144449818)","(8.0, 8.286565001487157)","(9.0, 8.560624043360434)"
4,"(nan, 1.4781798324457358)","(1.0, 1.8719600205810285)","(8.0, 8.368530416062397)","(3.0, 3.949388555408513)","(7.0, 5.474901027719156)"
5,"(2.0, 2.2828773872212613)","(3.0, 1.4486568604876837)","(5.0, 5.321090592177557)","(nan, 3.4616251620274174)","(nan, 4.2311978306738425)"
6,"(4.0, 3.315808956508669)","(2.0, 1.8105479985944275)","(nan, 6.181168783244281)","(2.0, 4.491013416363289)","(7.0, 5.285895298806231)"
7,"(7.0, 8.275398098554046)","(1.0, 2.200041683812154)","(2.0, 3.204507143902677)","(7.0, 6.9682078920038055)","(9.0, 6.401871764980757)"
8,"(3.0, 3.7169801741529636)","(3.0, 1.7592047023171675)","(nan, 5.5036737170400825)","(7.0, 4.539878278608792)","(3.0, 5.133531970752895)"
9,"(4.0, 3.0030664263863898)","(nan, 1.3798936989230564)","(5.0, 4.228236830199457)","(3.0, 3.5921579040820237)","(3.0, 4.026229289942425)"
